In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/afsis-soil-properties/train.zip')
test = pd.read_csv('/kaggle/input/afsis-soil-properties/test.zip')
sub = pd.read_csv('/kaggle/input/afsis-soil-properties/sample_submission.csv')

In [ ]:
def renameColumns(df, train=False):
    mx_col=[i for i in df.columns if 'm' in i ]
    mnotx_col=[i for i in df.columns if 'm' not in i ]
    mx_float_col=[float(midx[1:]) for midx in mx_col]
    mx_df=df[mx_col]
    mx_df.columns=mx_float_col
    first_der=[]
    for idx, row in mx_df.iterrows():
        first_der.append(row.diff().iloc[1:]/np.diff(row.index))
    first_der_df=pd.DataFrame(first_der)
    first_der_df.columns=mx_col[1:]
    mnotx_df=df[mnotx_col]
    mnotx_df.Depth=pd.factorize(mnotx_df.Depth)[0]
    
    if train:
        mnotx_df[['Ca']]
    else:
        Y=sub
    merge_df=pd.concat([first_der_df,mnotx_df], axis=1)
   
    return merge_df

In [ ]:
train_df=renameColumns(train)
test_df=renameColumns(test)

In [ ]:
test_df

In [ ]:
import lightgbm as lgb

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_df,random_state=42,test_size=0.1)

In [ ]:
y_list = list(train_df.columns)[-5:]
x_list = list(train_df.columns)[:-5]
x_list.remove('PIDN')
x1_list = [_ for _ in x_list if 'm' in str(_)]
x2_list = [_ for _ in x_list if 'm' not in str(_)]
print(x2_list, y_list)

In [ ]:
train_df.head()

In [ ]:
train_df.loc[:,x_list]

In [ ]:

# specify your configurations as a dict
params = {
    'task':'train',
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
#     'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 2,
    'num_iteration': 1000, 
    'early_stopping_round': 30,
    'max_depth':20,
    'categorical_feature':len(x_list)
}

In [ ]:
predict = {}
for y in y_list:
    # create dataset for lightgbm
    lgb_train = lgb.Dataset(train_df.loc[:,x_list], train_df.loc[:,y])
    lgb_eval = lgb.Dataset(val_df.loc[:,x_list], val_df.loc[:,y], reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval)
    predict[y] = gbm.predict(test_df.loc[:,x_list], num_iteration=gbm.best_iteration)

In [ ]:
pd.concat([test['PIDN'], pd.DataFrame(predict)], axis=1).to_csv('submit.csv', index=False)